In [1]:
import requests
import json

# Service URLS
StreamStatsServiceURLS = {
    'watershed': 'https://streamstats.usgs.gov/streamstatsservices/watershed.geojson',
    'basinCharacteristics': 'https://prodwebb.streamstats.usgs.gov/streamstatsservices/parameters.json'
    }
NSSServiceURlS = {
    'regressionRegions': 'https://streamstats.usgs.gov/nssservices/regressionregions/bylocation',
    'scenarios': 'https://streamstats.usgs.gov/nssservices/scenarios',
    'computeFlowStats': 'https://streamstats.usgs.gov/nssservices/scenarios/estimate'
}

# Declare cookies for use of consistent StreamStats servers
global cookies 

# Step 1 - Get the watershed
print('Step 1 - Get the watershed')
watershedURLParams = {
    'rcode': 'ID', 
    'xlocation': -115.32927582, 
    'ylocation': 44.90574095,
    'crs': 4326,
}
delineatedBasin = None
while delineatedBasin is None: # Sometimes the watershed endpoint returns with no geometry - this loop is written to keep trying until a geometry is returned
    print('Delineating basin...')
    basinDelineationResponse = requests.get(url = StreamStatsServiceURLS['watershed'], params = watershedURLParams)
    if basinDelineationResponse.status_code == 200:
        cookies = basinDelineationResponse.cookies
        try:
            delineatedBasin = json.loads(basinDelineationResponse.content.decode('utf-8'))["featurecollection"][1]["feature"]["features"][0]["geometry"] # this will be used in step 2
            workspaceID = json.loads(basinDelineationResponse.content.decode('utf-8'))["workspaceID"] # this will be used in step 4
        except:
            print("No geometry returned. Retrying...")
    else:
        print("Error. Retrying...")

# Step 2 - Get the Regression Regions associated with the watershed
print('Step 2 - Get the Regression Regions associated with the watershed')
regressionRegionPOSTBody = delineatedBasin # from step 1

regressionRegions = None
while regressionRegions == None:
    regressionRegionsResponse = requests.post(url = NSSServiceURlS['regressionRegions'], json=regressionRegionPOSTBody)
    if regressionRegionsResponse.status_code == 200:
        regressionRegions = json.loads(regressionRegionsResponse.content.decode('utf-8'))
        regressionRegionCodes = [ sub['code'] for sub in regressionRegions ] # this will be used in step 3
    else:
        print("Error.")

# Step 3 - Get the Scenarios associated with the Regression Regions
print('Step 3 - Get the Scenarios associated with the Regression Regions')

scenarioURLParams = {
    'regions': 'ID', 
    'statisticgroups': 2, # this can be updated depending on which flow statistics you want to calculate (2 is associated with Peak Flow Statistics) - you can find all statistic groups available for your region at https://streamstats.usgs.gov/docs/nssservices/#/StatisticGroups/GET/StatisticGroups
    'regressionregions': (','.join(regressionRegionCodes)) # from step 2, needs to be comma separated list
}
scenarioResponse = requests.get(url = NSSServiceURlS['scenarios'], params=scenarioURLParams)

parameters = None
while parameters == None:
    if scenarioResponse.status_code == 200:
        scenarios = json.loads(scenarioResponse.content.decode('utf-8'))[0] # this will be used in step 5
        parameters = json.loads(scenarioResponse.content.decode('utf-8'))[0]["regressionRegions"][0]["parameters"]
        parameterCodes = [ sub['code'] for sub in parameters ] # this will be used in step 4
    else:
        print("Error.")

# Step 4 - Compute the basin characteristics
print('Step 4 - Compute the basin characteristics')
basinCharacteristicsURLParams = {
    'rcode': 'ID', 
    'workspaceID': workspaceID, # from step 1
    'includeparameters': (','.join(parameterCodes)) # from step 3
}

basinCharacteristics = None
while basinCharacteristics is None:
    basinCharacteristicsResponse = requests.get(url = StreamStatsServiceURLS['basinCharacteristics'], params=basinCharacteristicsURLParams, cookies=cookies)
    if basinCharacteristicsResponse.status_code == 200:
        basinCharacteristics = json.loads(basinCharacteristicsResponse.content.decode('utf-8')) # this will be used in step 5
    else:
        print("Error.")

# Step 5 - Compute Flow Statistics
print('Step 5 - Compute Flow Statistics')
flowStatsURLParams = {
    'regions': 'ID'
}
flowStatsPOSTBody = [] 
for counter, x in enumerate(scenarios['regressionRegions'][0]['parameters']): # from step 3 & step 4
    for p in basinCharacteristics['parameters']:
        if x['code'].lower() == p['code'].lower():
            scenarios['regressionRegions'][0]['parameters'][counter]['value'] = p['value']
flowStatsPOSTBody.append(scenarios)

flowStats = None
while flowStats == None:
    flowStatsResponse = requests.post(url = NSSServiceURlS['computeFlowStats'], params = flowStatsURLParams, json=flowStatsPOSTBody)
    if flowStatsResponse.status_code == 200:
        flowStats = json.loads(flowStatsResponse.content.decode('utf-8'))
        print(flowStats)
    else:
        print("Error.")

Step 1 - Get the watershed
Delineating basin...
Step 2 - Get the Regression Regions associated with the watershed
Step 3 - Get the Scenarios associated with the Regression Regions
Step 4 - Compute the basin characteristics
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.
Error.


KeyboardInterrupt: 

In [4]:
flowStats

[{'statisticGroupID': 2,
  'statisticGroupName': 'Peak-Flow Statistics',
  'regressionRegions': [{'id': 719,
    'name': 'Peak_Region_3_2016_5118',
    'code': 'GC1752',
    'description': '',
    'statusID': 4,
    'citationID': 150,
    'parameters': [{'id': 19580,
      'name': 'Drainage Area',
      'description': 'Area that drains to a point on a stream',
      'code': 'DRNAREA',
      'unitType': {'id': 35, 'unit': 'square miles', 'abbr': 'mi^2'},
      'value': 0.0,
      'limits': {'max': 2610.0, 'min': 0.08}},
     {'id': 19581,
      'name': 'Mean Annual Precip PRISM 1981 2010',
      'description': 'Basin average mean annual precipitation for 1981 to 2010 from PRISM',
      'code': 'PRECPRIS10',
      'unitType': {'id': 36, 'unit': 'inches', 'abbr': 'in'},
      'value': 93.9,
      'limits': {'max': 168.0, 'min': 33.2}}],
    'results': [{'id': 0,
      'name': '50-percent AEP flood',
      'code': 'PK50AEP',
      'description': 'Maximum instantaneous flow that occurs with